# Introducción

En este proyecto, se aborda el problema de la evaluación de similitud semántica entre pares de oraciones. Este es un desafío fundamental en el procesamiento del lenguaje natural (NLP), con aplicaciones en tareas como recuperación de información, detección de plagio, sistemas de recomendación y más.

Este trabajo forma parte de la materia **Deep Learning para NLP**, donde se exploran técnicas avanzadas para resolver problemas complejos en el ámbito del lenguaje natural.

Para resolver este problema, se implementan y comparan tres enfoques basados en modelos de lenguaje preentrenados (BERT y variantes optimizadas para similitud semántica). Los modelos utilizados son:

1. **BERT + Regresión**: Un modelo basado en BERT con una capa de regresión para predecir la similitud.
2. **Siamese BERT**: Un modelo siamés que compara las representaciones de dos oraciones.
3. **Cross-Attention Model**: Un modelo que utiliza atención cruzada para capturar interacciones entre las oraciones.

El objetivo es entrenar y evaluar estos modelos en el conjunto de datos STS Benchmark, normalizando las etiquetas a un rango de 0 a 1. Finalmente, se validan los modelos con ejemplos de prueba para analizar su desempeño en diferentes niveles de similitud semántica.

In [80]:
import random
import numpy as np
import torch
import psutil
import os
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding, get_scheduler
from torch.optim import AdamW
from torch.utils.data import Dataset as TorchDataset, DataLoader
from torch.nn import Module
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer


Configuracion para la Reproducibilidad

In [81]:
# Seed para asegurar reproducibilidad sin importar el dispositivo en el que se ejecute
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

Configurando el Dispositivo

In [82]:
# Configurar dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Carga y Preprocesamiento del Dataset

In [83]:
# Cargar dataset (STS-B o personalizado)
def load_custom_dataset(csv_path=None):
    if csv_path:
        # Cargar dataset personalizado desde CSV
        df = pd.read_csv(csv_path)
        assert all(col in df.columns for col in ['sentence1', 'sentence2', 'score']), "CSV debe tener columnas: sentence1, sentence2, score"
        dataset = Dataset.from_pandas(df)
    else:
        # Usar dataset de ejemplo (tecnología) si no se proporciona CSV
        data = {
            'sentence1': [
                "The new smartphone has a 120Hz display.",
                "AI algorithms improve processing speed.",
                "The laptop features a high-capacity SSD.",
                "Cloud computing enhances scalability.",
                "Quantum computers solve complex problems."
            ],
            'sentence2': [
                "This phone offers a smooth 120Hz screen.",
                "Machine learning boosts performance.",
                "The notebook includes a fast SSD drive.",
                "Cloud services improve flexibility.",
                "Traditional computers handle simple tasks."
            ],
            'score': [4.8, 4.5, 4.7, 4.6, 2.0]
        }
        dataset = Dataset.from_dict(data)
    # Normalizar puntuaciones a [0, 1]
    def normalize_labels(examples):
        examples["score"] = [s / 5.0 for s in examples["score"]]
        return examples
    dataset = dataset.map(normalize_labels, batched=True)
    return dataset

# Cargar dataset (cambiar csv_path a tu archivo CSV si tienes uno)
csv_path = None  # Ejemplo: "path/to/your/dataset.csv"
dataset = load_dataset("mteb/stsbenchmark-sts") if csv_path is None else load_custom_dataset(csv_path)



Tokenizacion del Texto

In [84]:
#Creamos un tokenizador para convertir las frases en IDs de tokens que el modelo pueda procesar. Usare BERT o un modelo optimizado para similitud semántica
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-MiniLM-L6-v2")

#Definiendo la funcion de tokenizacion
def tokenize_function(examples):
    #Tokenizando sentence1 y sentence2 por separado
    #Esto es necesario porque el modelo espera que las entradas sean pares de oraciones
    #y no una sola oración.
    encodings1 = tokenizer(
        examples['sentence1'],
        padding =False,
        truncation=True,
        max_length=128,
        return_tensors=None # Lo dejo como lista para el dataset
    )
    encodings2 = tokenizer(
        examples['sentence2'],
        padding=False,
        truncation=True,
        max_length=128,
        return_tensors=None # Lo dejo como lista para el dataset
    )
    return{
        'input_ids1': encodings1['input_ids'],
        'attention_mask1': encodings1['attention_mask'],
        'input_ids2': encodings2['input_ids'],
        'attention_mask2': encodings2['attention_mask'],
        'score': examples['score'] # Mantengo la etiqueta original
    }
    
#Tokenizando el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/5749 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1379 [00:00<?, ? examples/s]

Probando 123...

In [85]:
# Verificar longitudes máximas
max_len1 = max(len(x) for x in tokenized_datasets['train']['input_ids1'])
max_len2 = max(len(x) for x in tokenized_datasets['train']['input_ids2'])
print(f"Longitud máxima input_ids1: {max_len1}")
print(f"Longitud máxima input_ids2: {max_len2}")

Longitud máxima input_ids1: 70
Longitud máxima input_ids2: 63


Creando el Dataset para Pytorch

In [86]:
#Creo una clase para formatear los datos de entrada correctamente para PyTorch
class STSDataset(TorchDataset):
    def __init__(self, data):
        self.input_ids1 = data["input_ids1"]
        self.attention_mask1 = data["attention_mask1"]
        self.input_ids2 = data["input_ids2"]
        self.attention_mask2 = data["attention_mask2"]
        self.scores = data["score"]
    def __len__(self):
        return len(self.input_ids1)
    def __getitem__(self, idx):
        return {
            'input_ids1': self.input_ids1[idx],
            'attention_mask1': self.attention_mask1[idx],
            'input_ids2': self.input_ids2[idx],
            'attention_mask2': self.attention_mask2[idx],
            'labels': self.scores[idx]
        }

# Dividir dataset
if 'train' in tokenized_datasets:
    train_dataset = STSDataset(tokenized_datasets["train"])
    val_dataset = STSDataset(tokenized_datasets["validation"])
    test_dataset = STSDataset(tokenized_datasets["test"])
else:
    # Si es un dataset personalizado, dividir manualmente
    dataset = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
    train_val = dataset['train'].train_test_split(test_size=0.25, seed=42)  # 60% train, 20% val, 20% test
    train_dataset = STSDataset(train_val['train'])
    val_dataset = STSDataset(train_val['test'])
    test_dataset = STSDataset(dataset['test'])

Creando un DataCollator Customizado

In [87]:
class CustomDataCollatorWithPadding:
    def __init__(self, tokenizer, max_length=128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.padding_collator = DataCollatorWithPadding(tokenizer, max_length=max_length)
    
    def __call__(self, examples):
        features1 = [
            {'input_ids': ex['input_ids1'], 'attention_mask': ex['attention_mask1']}
            for ex in examples
        ]
        features2 = [
            {'input_ids': ex['input_ids2'], 'attention_mask': ex['attention_mask2']}
            for ex in examples
        ]
        labels = [ex['labels'] for ex in examples]
        batch1 = self.padding_collator(features1)
        batch2 = self.padding_collator(features2)
        batch = {
            'input_ids1': batch1['input_ids'],
            'attention_mask1': batch1['attention_mask'],
            'input_ids2': batch2['input_ids'],
            'attention_mask2': batch2['attention_mask'],
            'labels': torch.tensor(labels, dtype=torch.float32)
        }
        return batch

data_collator = CustomDataCollatorWithPadding(tokenizer, max_length=128)


# Creando los DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=data_collator)

# Para verificar formas del batch
for batch in train_loader:
    print(f"input_ids1 shape: {batch['input_ids1'].shape}")
    print(f"input_ids2 shape: {batch['input_ids2'].shape}")
    break


input_ids1 shape: torch.Size([8, 23])
input_ids2 shape: torch.Size([8, 18])


### Definiendo los Modelos

Modelo 1 : Siamese BERT

In [88]:
# Definiendo el primer modelo a utilizar, en este caso estaremos utilizando el modelo de 
# Sentence Transformers "all-MiniLM-L6-v2".
# Este modelo es un modelo de transformador optimizado para tareas de similitud semántica.
# Se congela un número configurable de capas iniciales del modelo BERT para reducir el costo computacional.
# La salida de las dos oraciones se concatena y pasa por una red neuronal para predecir la similitud.
class SentenceSimilarityModelOne(Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=2):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state[:, 0, :]
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]
        combined = torch.cat([output1, output2], dim=1)
        return self.regressor(combined).squeeze()


Modelo 2: Siamese BERT

In [89]:
# Definiendo el segundo modelo a utilizar, este modelo es una variante de BERT conocida como Siamese BERT.
# Siamese BERT utiliza dos instancias del modelo BERT para procesar dos oraciones de forma independiente.
# Luego, combina las representaciones de las oraciones utilizando operaciones como la diferencia absoluta y el producto elemento a elemento.
# Finalmente, una red neuronal realiza la regresión para predecir la similitud semántica entre las oraciones.
class SiameseBERT(Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=0):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        # Congelar capas según parámetro freeze_layers
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 3, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1)
        )
    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state[:, 0, :]
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state[:, 0, :]
        diff = torch.abs(output1 - output2)
        mult = output1 * output2
        combined = torch.cat([diff, mult, output1], dim=1)
        return self.regressor(combined).squeeze()


Modelo 3: Cross-Attention Model

In [90]:
# Definiendo el tercer modelo a utilizar, este modelo utiliza atención cruzada para capturar interacciones entre las representaciones de dos oraciones.
# Se basa en un modelo de transformador preentrenado (paraphrase-MiniLM-L6-v2) y aplica atención cruzada para combinar las representaciones de las oraciones.
# Finalmente, utiliza una red neuronal para predecir la similitud semántica entre las oraciones.
class CrossAttentionModel(Module):
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2", freeze_layers=2):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        for i, param in enumerate(self.bert.encoder.layer):
            if i < (len(self.bert.encoder.layer) - freeze_layers):
                for p in param.parameters():
                    p.requires_grad = False
        self.attention = torch.nn.MultiheadAttention(embed_dim=384, num_heads=8)
        self.regressor = torch.nn.Sequential(
            torch.nn.Linear(384 * 2, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 1),
            torch.nn.Sigmoid()
        )
    def forward(self, input_ids1, attention_mask1, input_ids2, attention_mask2):
        output1 = self.bert(input_ids1, attention_mask=attention_mask1).last_hidden_state
        output2 = self.bert(input_ids2, attention_mask=attention_mask2).last_hidden_state
        mask1 = attention_mask1.unsqueeze(-1).expand_as(output1)
        mask2 = attention_mask2.unsqueeze(-1).expand_as(output2)
        output1 = (output1 * mask1).sum(dim=1) / mask1.sum(dim=1)
        output2 = (output2 * mask2).sum(dim=1) / mask2.sum(dim=1)
        output1 = output1.unsqueeze(0)
        output2 = output2.unsqueeze(0)
        attn_output, _ = self.attention(output1, output2, output2)
        attn_output = attn_output.squeeze(0)
        combined = torch.cat([output1.squeeze(0), attn_output], dim=1)
        return self.regressor(combined).squeeze()


### Configuracion del Entrenamiento

In [91]:

# Inicializando los modelos
model1 = SentenceSimilarityModelOne().to(device)
model2 = SiameseBERT().to(device)
model3 = CrossAttentionModel().to(device)

# Definiendo la  función de pérdida de correlación
def pearson_correlation_loss(outputs, targets):
    outputs = outputs - torch.mean(outputs)
    targets = targets - torch.mean(targets)
    norm_outputs = torch.sqrt(torch.sum(outputs ** 2))
    norm_targets = torch.sqrt(torch.sum(targets ** 2))
    correlation = torch.sum(outputs * targets) / (norm_outputs * norm_targets)
    return 1 - correlation


Validando el Fine - Tuning de los Modelos

In [92]:
def print_trainable_params(model):
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f"Total Parámetros: {total_params:,}, Entrenables: {trainable_params:,}")

print("\nBERT + Regresión:")
print_trainable_params(model1)

print("\nSiamese BERT:")
print_trainable_params(model2)

print("\nCross-Attention:")
print_trainable_params(model3)



BERT + Regresión:
Total Parámetros: 22,910,337, Entrenables: 15,812,481

Siamese BERT:
Total Parámetros: 23,008,641, Entrenables: 12,361,857

Cross-Attention:
Total Parámetros: 23,501,697, Entrenables: 16,403,841


Monitoreo de los recursos

### Evaluación del Modelo

In [93]:
# Función de evaluación
def evaluate_model(model, val_loader, model_type="bert"):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
        for batch in val_loader:
            inputs1 = batch["input_ids1"].to(device)
            mask1 = batch["attention_mask1"].to(device)
            inputs2 = batch["input_ids2"].to(device)
            mask2 = batch["attention_mask2"].to(device)
            lbls = batch["labels"].to(device)
            outputs = model(inputs1, mask1, inputs2, mask2)
            preds.extend(outputs.cpu().numpy())
            labels.extend(lbls.cpu().numpy())
    correlation, _ = pearsonr(preds, labels)
    return correlation

In [94]:
# def weighted_mse_loss(output, target):
#     weights = torch.where(target < 1,2.0,1.0) # para similitudes muy bajas, se penaliza más
#     return torch.mean(weights * (output - target) ** 2)

# Función para entrenar modelo con diferentes configuraciones
def train_model(model, train_loader, val_loader, model_type="bert", epochs=3, lr=3e-5, output_dir="./checkpoints"):
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * epochs)
    history = []
    best_pearson = -float("inf")
    best_model_path = os.path.join(output_dir, f"best_model_{model_type}.pt")
    os.makedirs(output_dir, exist_ok=True)
    for epoch in range(epochs):
        total_loss = 0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs1 = batch["input_ids1"].to(device)
            mask1 = batch["attention_mask1"].to(device)
            inputs2 = batch["input_ids2"].to(device)
            mask2 = batch["attention_mask2"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(inputs1, mask1, inputs2, mask2)
            if model_type == "siamese":
                loss = pearson_correlation_loss(outputs, labels)
            else:
                loss = torch.nn.MSELoss()(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            total_loss += loss.item()
            torch.cuda.empty_cache()
        avg_train_loss = total_loss / len(train_loader)
        pearson_corr = evaluate_model(model, val_loader, model_type)
        history.append((avg_train_loss, pearson_corr))
        print(f"{model_type.upper()} - Epoch {epoch+1}/{epochs} - Loss: {avg_train_loss:.4f}, Pearson: {pearson_corr:.4f}")
        if pearson_corr > best_pearson:
            best_pearson = pearson_corr
            torch.save(model.state_dict(), best_model_path)
            print(f"Mejor modelo guardado con correlación de Pearson: {best_pearson:.4f}")
    model.load_state_dict(torch.load(best_model_path))
    print(f"{model_type.upper()} - Entrenamiento finalizado. Mejor modelo cargado: {best_pearson:.4f}")
    return history, best_pearson

def evaluate_on_test(model, test_loader, model_type="bert"):
    person_corr = evaluate_model(model, test_loader, model_type)
    print(f"{model_type.upper()} - Evaluación en test - Pearson Correlation: {person_corr:.4f}")
    return person_corr

In [95]:
# Función de inferencia
def infer_similarity(model, tokenizer, sentence1, sentence2):
    model.eval()
    with torch.no_grad():
        encoded1 = tokenizer(
            sentence1,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)
        encoded2 = tokenizer(
            sentence2,
            padding="max_length",
            truncation=True,
            max_length=128,
            return_tensors="pt"
        ).to(device)
        similarity = model(
            encoded1["input_ids"],
            encoded1["attention_mask"],
            encoded2["input_ids"],
            encoded2["attention_mask"]
        )
        # Normalizar salida a [0, 5]
        if isinstance(model, SiameseBERT):
            similarity = torch.tanh(similarity) * 2.5 + 2.5  # Mapear a [0, 5]
        else:
            similarity = similarity * 5
    return similarity.item()


In [96]:
# Entrenar y evaluar modelos
models = {
    "BERT + Regresión": (model1, "bert"),
    "Siamese BERT": (model2, "siamese"),
    "Cross-Attention": (model3, "cross")
}
test_results = {}
for model_name, (model, model_type) in models.items():
    print(f"\nEntrenando {model_name}...")
    history, best_pearson = train_model(model, train_loader, val_loader, model_type=model_type)
    test_results[model_name] = best_pearson
    print(f"Correlación de Pearson en validación para {model_name}: {best_pearson:.4f}")



Entrenando BERT + Regresión...
BERT - Epoch 1/3 - Loss: 5.1189, Pearson: 0.1707
Mejor modelo guardado con correlación de Pearson: 0.1707
BERT - Epoch 2/3 - Loss: 5.0379, Pearson: 0.1716
Mejor modelo guardado con correlación de Pearson: 0.1716
BERT - Epoch 3/3 - Loss: 5.0379, Pearson: 0.1718
Mejor modelo guardado con correlación de Pearson: 0.1718
BERT - Entrenamiento finalizado. Mejor modelo cargado: 0.1718
Correlación de Pearson en validación para BERT + Regresión: 0.1718

Entrenando Siamese BERT...
SIAMESE - Epoch 1/3 - Loss: 0.3508, Pearson: 0.8182
Mejor modelo guardado con correlación de Pearson: 0.8182
SIAMESE - Epoch 2/3 - Loss: 0.2729, Pearson: 0.8240
Mejor modelo guardado con correlación de Pearson: 0.8240
SIAMESE - Epoch 3/3 - Loss: 0.2439, Pearson: 0.8251
Mejor modelo guardado con correlación de Pearson: 0.8251
SIAMESE - Entrenamiento finalizado. Mejor modelo cargado: 0.8251
Correlación de Pearson en validación para Siamese BERT: 0.8251

Entrenando Cross-Attention...
CROSS -

In [97]:
# Evaluar en ejemplos de prueba
test_sentences = [
    ("I love eating apples", "The capital of France is Paris"),
    ("I have a black cat", "My pet is a dog"),
    ("He plays soccer on weekends", "She enjoys playing tennis on Sundays"),
    ("The sun is shining in the sky", "It is a bright and sunny day"),
    ("The smartphone has a large screen", "This phone features a big display"),
    ("The dog is barking loudly", "The dog is barking loudly")
]
for model_name, (model, model_type) in models.items():
    print(f"\nEvaluando {model_name} en ejemplos de prueba:")
    for i, (s1, s2) in enumerate(test_sentences):
        score = infer_similarity(model, tokenizer, s1, s2)
        print(f"Ejemplo {i+1}: {s1} | {s2} → Score: {score:.2f} --> {round(score)}")



# Comparar con modelo preentrenado
pretrained_model = SentenceTransformer("paraphrase-MiniLM-L6-v2")
print("\nEvaluando modelo preentrenado en ejemplos de prueba:")
for i, (s1, s2) in enumerate(test_sentences):
    embeddings1 = pretrained_model.encode(s1)
    embeddings2 = pretrained_model.encode(s2)
    similarity = np.dot(embeddings1, embeddings2) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings2))
    score = similarity * 5
    print(f"Ejemplo {i+1}: {s1} | {s2} → Score: {score:.2f} --> {round(score)}")


Evaluando BERT + Regresión en ejemplos de prueba:
Ejemplo 1: I love eating apples | The capital of France is Paris → Score: 5.00 --> 5
Ejemplo 2: I have a black cat | My pet is a dog → Score: 5.00 --> 5
Ejemplo 3: He plays soccer on weekends | She enjoys playing tennis on Sundays → Score: 5.00 --> 5
Ejemplo 4: The sun is shining in the sky | It is a bright and sunny day → Score: 5.00 --> 5
Ejemplo 5: The smartphone has a large screen | This phone features a big display → Score: 5.00 --> 5
Ejemplo 6: The dog is barking loudly | The dog is barking loudly → Score: 5.00 --> 5

Evaluando Siamese BERT en ejemplos de prueba:
Ejemplo 1: I love eating apples | The capital of France is Paris → Score: 0.83 --> 1
Ejemplo 2: I have a black cat | My pet is a dog → Score: 1.43 --> 1
Ejemplo 3: He plays soccer on weekends | She enjoys playing tennis on Sundays → Score: 1.37 --> 1
Ejemplo 4: The sun is shining in the sky | It is a bright and sunny day → Score: 2.00 --> 2
Ejemplo 5: The smartphone has 

In [98]:

for model_name, (model, model_type) in models.items():
    print(f"\nEntrenando {model_name} en el conjunto de test...")
    test_pearson = evaluate_on_test(model, test_loader, model_type)
    test_results[model_name] = test_pearson
    print(f"Correlación de Pearson en test para {model_name}: {test_pearson:.4f}")


Entrenando BERT + Regresión en el conjunto de test...
BERT - Evaluación en test - Pearson Correlation: 0.0060
Correlación de Pearson en test para BERT + Regresión: 0.0060

Entrenando Siamese BERT en el conjunto de test...
SIAMESE - Evaluación en test - Pearson Correlation: 0.7832
Correlación de Pearson en test para Siamese BERT: 0.7832

Entrenando Cross-Attention en el conjunto de test...
CROSS - Evaluación en test - Pearson Correlation: -0.0035
Correlación de Pearson en test para Cross-Attention: -0.0035
